<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/virt_udpipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install spacy_udpipe
import spacy_udpipe

In [3]:
%%capture
!pip install pymorphy2[fast]
from pymorphy2 import MorphAnalyzer 
morph = MorphAnalyzer()

In [42]:
import os
import unicodedata
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import Counter, defaultdict
from sklearn.metrics.pairwise import cosine_similarity

# Sentence-level experiment

For further exploration of the semantic axis method we make use of BERT embeddings.

Seed verbs remain the same as in the word-level experiments, but may be subject to change later.

1.   We construct seed sentences with regard to the arguments of the test sentence. For example, to test a sentence like 'Силовики вломились к журналисту', we compute the semantic axis as follows:

*   Replace the target verb with a seed verb.
*   Make changes to cases of arguments if necessary.
*   Repeat for each seed verb to construct seed sentences.

2.   We embed sentences in the two seed groups and take averages of the two corresponding embedding groups.

3.   We compute semantic axis for the test sentense by substracting the embedding of the negative seed from embedding of the positive seed.

4.   We embed the test sentense and measure it's cosine similarity to the axis

5.   We repeat previous steps for each test sentence with the predicate in question.

6. The resulting dataset for all test sentenses will hopefully look like this:

| Predicate(verb)      | Polarity | Text     | Similarity     |
|    :----   |    :----   |    :----   |    ----:   |
| защищать      | pos       | он защищает его   | 0,333       |
| защищать      | pos       | суд защищает права   | 0,321       |


# BERT-as-service

Check out [this issue](https://github.com/hanxiao/bert-as-service/issues/380) and "make sure Colab is using Tensorflow 1.x, because bert-serving-start doesn't currently work with TF 2.1 and nohup hides the output of the command failing".

Also make sure you're using GPU accelerator.

In [ ]:
%tensorflow_version 1.x
# import tensorflow as tf
# print (tf.__version__)

TensorFlow 1.x selected.


In [ ]:
%%capture
!pip install -U bert-serving-server[http]
!pip install bert-serving-client  # client, independent of `bert-serving-server`

In [ ]:
%%capture
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip /content/multi_cased_L-12_H-768_A-12.zip

In [ ]:
!nohup bert-serving-start -model_dir=./multi_cased_L-12_H-768_A-12 > out.file 2>&1 &

In [ ]:
from bert_serving.client import BertClient
bc = BertClient()

In [ ]:
encoded_test = bc.encode(['First do it', 
                          'then do it right', 'then do it better'
                          ])
len(encoded_test[0])

768

# UDPipe

In [12]:
spacy_udpipe.download("ru")
syntagrus = spacy_udpipe.load("ru")

Downloaded pre-trained UDPipe model for 'ru' language


In [ ]:
# with open('16.txt') as f:
#     text = f.read()

# verbs = list(pd.read_csv('cross_seminar_task.csv', sep='\t')['verb'])

def text2ud(text):
    udpiped = []
    doc = syntagrus(text)
    doc_len = len(doc)
    for i, token in enumerate(doc):
        if i <= 10 or i == doc_len-10:
            continue
        if token.lemma_ in verbs:
            new_entry = {token.lemma_: []}
            for t in reversed(doc[i-10:i]):
                if t.head.lemma_ == token.lemma_:
                    new_entry[token.lemma_].append([t.text, t.lemma_, t.pos_, t.dep_])
            for t in doc[i:i+10]:
                if t.head.lemma_ == token.lemma_:
                    new_entry[token.lemma_].append([t.text, t.lemma_, t.pos_, t.dep_])
            udpiped.append(new_entry)
            
    with open('result.json', 'w') as f:
        json.dump(udpiped, f, ensure_ascii=False, indent=4)

# text2ud(text[:1000])

# Semantic axis method

In [ ]:
class SemanticAxis():

    def __init__(self):
        self.seed0 = set()
        self.seed1 = set()
        self.targets = set()
        self.axis_vector = None
        self.axis_similarities = None

    def add_seed(self, seed: set, seed_id: int):
        if seed_id:
            self.seed1 = set(seed)
        else:
            self.seed0 = set(seed)
    def add2seed(self, seed: set, seed_id: int):
        if seed_id:
            self.seed1.update(seed)
        else:
            self.seed0.update(seed)
    def flush_seed(self, seed_id=None, flush_both_seeds=True):
        if seed_id != None:
            if seed_id:
                self.seed1 = set()
            else:
                self.seed0 = set()
        else:
            self.seed0, self.seed1 = set(), set()
    
    def add_targets(self, target):
        self.targets = target
    def add2targets(self, target):
        self.targets.update(target)
    def flush_targets(self):
        self.targets = set()
    
    def compute_bert_axis(self, bert_client):
        assert len(self.seed0) > 0, 'Seed0 set is empty.'
        assert len(self.seed1) > 0, 'Seed1 set is empty.'
        self.bert_client = bert_client

        target_vectors = self.bert_client.encode(list(self.targets))
        seed_vectors = [self.bert_client.encode(list(s)).mean(axis=0) 
        for s in (self.seed0, self.seed1)]

        self.axis_vector = seed_vectors[1] - seed_vectors[0]

        self.axis_similarities = {self.targets[i]:cosine_similarity(
            np.atleast_2d(vector), 
            np.atleast_2d(self.axis_vector)
            ).item() for i, vector in enumerate(target_vectors)}

In [ ]:
sa = SemanticAxis()

In [ ]:
sa.add_targets(list2check)
sa.add_seed(['разрушать'], 0)
sa.add_seed(['ценить'], 1)

In [ ]:
sa.compute_bert_axis(bc)
df = pd.DataFrame({'target':sa.axis_similarities.keys(),
                   'similarity':sa.axis_similarities.values()})
df.sort_values(by='similarity')

# Triples (nsubj, root, obj)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unrar x ""

In [6]:
def ovd_loader():
    path = 'OVD-Info/2019'
    for month in os.listdir(path):
        for filename in os.listdir('{}/{}'.format(path, month)):
            filepath = '{}/{}/{}'.format(path, month, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                yield filepath, f.read()

In [7]:
d = {x[0]:unicodedata.normalize("NFKD", x[1]) for x in ovd_loader()}
texts = pd.DataFrame({'url':d.keys(),
                      'text':d.values()})

In [198]:
texts['spacy_doc'] = texts.text.progress_apply(syntagrus)

100%|██████████| 3073/3073 [17:20<00:00,  2.95it/s]


# WIP

In [187]:
# переписать для spacy docs
def triples_mapping(text, tags=tags, udpiper=syntagrus):
    doc = udpiper(text)

    for sent in doc.sents:
        sent_triples = []
        root_triple = defaultdict(list)
        # переписать, оставить леммы и pos-теги
        root_triple['root'] = sent.root
        for child in sent.root.children:
            if child.dep_ in tags:
                root_triple[child.dep_].append(child)
        sent_triples.append(dict(root_triple))
    return sent_triples

In [189]:
# tqdm.pandas()
# tags = 'nsubj obj iobj'.split()
# texts['triples'] = texts.text.progress_apply(triples_mapping)

100%|██████████| 3073/3073 [17:13<00:00,  2.97it/s]


In [201]:
pd.to_pickle(texts.spacy_doc, 'ovd-info_spacy_docs.pkl')

In [86]:
triple_set = Counter()

# переписать для колонки с spacy docs
for text, text_triples in tripled.items():
    for sent_triples in text_triples:
        d = sent_triples[0]
        # выкинули предложения с несколькими iobj
        if 'iobj' in d:
            if len(d['iobj']) > 1:
                continue
        # выкинули предложения с несколькими nsubj
        if 'nsubj' in d:
            if len(d['nsubj']) > 1:
                continue
        # переписать, потому что iobj остаются за бортом
            else:
                if 'obj' in d:
        # для каждого дополнения
                    for obj in d['obj']:
                        tup = tuple(map(str, (d['nsubj'][0], d['root'], obj)))
                        triple_set.update({tuple(t.lower() for t in tup)})

In [ ]:
triple_set.most_common(200)

In [92]:
with open ('tripled_ovd-info_2019.json', 'w', encoding='utf-8') as f:
    json.dump({' '.join(list(tr)):v for tr, v in triple_set.items()}, 
        f, ensure_ascii=False)

In [148]:
predicates = {}
predicates = defaultdict(list)
for tr, v in triple_set.most_common():
    for var in morph.parse(tr[1]):
        if ('VERB' in var.tag) or ('INFN' in var.tag):
            predicates[var.normal_form].append((tr, v))
            continue

In [128]:
!wget 'https://github.com/DmitryKutsev/NIS_SentiFrame/raw/master/annotations/polarity_annotation/3annotators_agree_on_these.zip'
!unzip '/content/3annotators_agree_on_these.zip'
with open('annotated_negative_3annotators_agree.json')as f:
    nverbs = set(json.load(f))
with open('annotated_positive_3annotators_agree.json')as f:
    pverbs = set(json.load(f))

In [ ]:
p_multi_seed = ['одобрять', 'хвалить', 'поощрять', 'любить',
                'обожать', 'восхищаться', 'восторгаться', 'гордиться',
                'ценить','превозносить',]
n_multi_seed = ['ненавидеть', 'ругать', 'злиться', 'порицать', 
                'осуждать', 'негодовать', 'обвинять', 'наказывать', 
                'убивать', 'разрушать',]

In [155]:
test_0 = 'избить задержать арестовать'.split()
test_1 = 'поддержать оправдать защищать'.split()
test_0 + test_1

['избить', 'задержать', 'арестовать', 'поддержать', 'оправдать', 'защищать']

In [156]:
test_triples = defaultdict(list)

for v in test_0 + test_1:
    for triple in predicates[v]:
        test_triples[v].append(triple[0])

In [ ]:
def (triple, seed0, seed1, morph=morph):
    psents = []
    n_sents = []
    nsubj, root, obj = tuple(map(morph.parse, triple))
    root.tag
    

In [ ]:
morph.parse('предстояло')[0].inflect({'pres', '3per'})
dir(morph.parse('предстояло')[0].tag)